In [3]:
import findspark
import os
spark_home = os.path.abspath("/Users/yanji/opt/spark")
findspark.init(spark_home)
import pyspark

os.system("kill -9 $(lsof -i:4050 -t) 2> /dev/null")

spark = pyspark.sql.SparkSession.builder \
    .master("local") \
    .appName("test") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print("http://{}:{}".format(spark.conf.get("spark.driver.bindAddress"), spark.conf.get("spark.ui.port")))


22/05/19 15:14:43 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

http://localhost:4050


## Spark-submit
Submit codes to production

## Datasets, Type-safe structured APIs
easier to cooperate for large projects
not available for python and r, only with Java and Scala


## Structured Streaming
high-level API for steaming process

In [4]:
sdf = spark.read.format("csv") \
    .option("header", "true") \
        .option("inferSchema", "true") \
            .load("../data/retail-data/by-day/*.csv")

In [5]:
sdf.createOrReplaceTempView("retail_data")
staticSchema = sdf.schema

In [6]:
from pyspark.sql.functions import window, column, desc, col

sdf.selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost","InvoiceDate") \
    .groupBy( col("CustomerId"), window(col("InvoiceDate"), "1 day") ) \
        .sum("total_cost").sort(desc("sum(total_cost)")).show()

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17450.0|{2011-09-20 08:00...|          71601.44|
|      null|{2011-11-14 08:00...|          55316.08|
|      null|{2011-11-07 08:00...|          42939.17|
|      null|{2011-03-29 08:00...| 33521.39999999998|
|      null|{2011-12-08 08:00...|31975.590000000007|
|   18102.0|{2011-09-15 08:00...|31661.540000000005|
|      null|{2010-12-21 08:00...|31347.479999999938|
|   18102.0|{2011-10-21 08:00...|          29693.82|
|   18102.0|{2010-12-07 08:00...|          25920.37|
|   14646.0|{2011-10-20 08:00...|25833.559999999994|
|      null|{2010-12-10 08:00...|25399.560000000012|
|      null|{2010-12-17 08:00...|25371.769999999768|
|      null|{2011-11-25 08:00...|24148.069999999992|
|      null|{2011-11-29 08:00...|23744.250000000055|
|   12415.0|{2011-06-15 08:00...| 23426.81000000001|
|      null|{2010-12-06 08:00...|23395.0999999

In [7]:
steamingdf = spark.readStream \
    .schema(staticSchema) \
    .option("maxFilesPerTrigger", 1) \
    .format("csv") \
    .option("header", "true") \
    .load("../data/retail-data/by-day/*.csv")

In [8]:
steamingdf.isStreaming

True

In [9]:
purchaseByCustomerPerhour = steamingdf \
    .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
    .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day")) \
    .sum("total_cost")

In [10]:
purchaseByCustomerPerhour.writeStream.format("memory") \
    .queryName("customer_purchase") \
    .outputMode("complete") \
    .start()

22/05/19 15:15:03 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/5p/b3c12d_s2nq3_m4_vw5_9gkh0000gn/T/temporary-b975a075-cfee-4101-abf3-367981844dc5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/19 15:15:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [14]:
spark.sql("SELECT * FROM customer_purchase").show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   14079.0|{2011-08-07 08:00...|375.09999999999997|
|   14621.0|{2011-06-05 08:00...|330.60999999999996|
|   16989.0|{2011-07-12 08:00...|             857.7|
|   17560.0|{2011-05-19 08:00...|            323.95|
|   17955.0|{2011-05-04 08:00...|270.34999999999997|
+----------+--------------------+------------------+
only showing top 5 rows



22/05/19 15:18:30 WARN TaskSetManager: Stage 654 contains a task of very large size (1399 KiB). The maximum recommended task size is 1000 KiB.
22/05/19 15:20:51 WARN FileStreamSource: Listed 305 file(s) in 2006 ms          
22/05/19 15:20:53 WARN FileStreamSource: Listed 305 file(s) in 2025 ms          
22/05/19 15:22:03 WARN FileStreamSource: Listed 305 file(s) in 2040 ms          


In [15]:
sdf.printSchema()    

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [16]:
from pyspark.sql.functions import date_format, col

prepped_df = sdf.na.fill(0) \
    .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE")) \
    .coalesce(5)

In [23]:
prepped_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|     Monday|
|   580538|    21544|SKULLS  WATER TRA..

In [20]:
trainDataFrame = prepped_df\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = prepped_df\
  .where("InvoiceDate >= '2011-07-01'")

In [22]:
trainDataFrame.count(), testDataFrame.count()

22/05/19 15:27:54 WARN FileStreamSource: Listed 305 file(s) in 2372 ms          


(245903, 296006)

22/05/19 15:27:57 WARN FileStreamSource: Listed 305 file(s) in 2417 ms


In [27]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
    .setInputCol("day_of_week")\
    .setOutputCol("day_of_week_index")

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
    .setInputCol("day_of_week_index")\
    .setOutputCol("day_of_week_encoded")


In [32]:
from pyspark.ml.feature import VectorAssembler
vector = VectorAssembler()\
    .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
    .setOutputCol("features")

from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
    .setStages([indexer, encoder, vector])

In [35]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

22/05/19 15:39:31 WARN FileStreamSource: Listed 305 file(s) in 3280 ms          


In [39]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [37]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [41]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(20).setSeed(1)

In [42]:
kmModel = kmeans.fit(transformedTraining)

22/05/19 15:42:21 WARN FileStreamSource: Listed 305 file(s) in 6389 ms          
22/05/19 15:42:38 WARN FileStreamSource: Listed 305 file(s) in 3645 ms          


In [ ]:
transformedTest = fittedPipeline.transform(testDataFrame)
transformedTest.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_index|day_of_week_encoded|            features|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|              2.0|      (5,[2],[1.0])|(7,[0,1,4],[1.79,...|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|              2.0|      (5,[2],[1.0])|(7,[0,1,4],[1.25,...|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdo

22/05/19 16:15:35 WARN FileStreamSource: Listed 305 file(s) in 3717 ms          
22/05/19 16:15:49 WARN FileStreamSource: Listed 305 file(s) in 5250 ms          
22/05/19 16:15:57 WARN FileStreamSource: Listed 305 file(s) in 8020 ms          
22/05/19 16:16:04 WARN FileStreamSource: Listed 305 file(s) in 7347 ms          
22/05/19 16:16:11 WARN FileStreamSource: Listed 305 file(s) in 6472 ms          
22/05/19 16:16:18 WARN FileStreamSource: Listed 305 file(s) in 6657 ms          
22/05/19 16:16:24 WARN FileStreamSource: Listed 305 file(s) in 6560 ms          
22/05/19 16:32:40 WARN FileStreamSource: Listed 305 file(s) in 7848 ms          
22/05/19 16:32:57 WARN FileStreamSource: Listed 305 file(s) in 6480 ms          
22/05/19 16:33:03 WARN FileStreamSource: Listed 305 file(s) in 6793 ms          
22/05/19 16:33:10 WARN FileStreamSource: Listed 305 file(s) in 6576 ms          
22/05/19 16:33:16 WARN FileStreamSource: Listed 305 file(s) in 6336 ms          
22/05/19 16:33:23 WARN FileS